# 🧠 NSSS Security Auditor - Colab Runner (Inference Only)

This notebook deploys the Neuro-Symbolic Software Security (NSSS) Auditor server on Google Colab.
It is designed for **inference** using either a locally fine-tuned model or a remote provider (Gemini/OpenAI).

**Features:**
- **Auto-Tunneling:** Exposes the server via Ngrok.
- **Robust Setup:** Auto-repairing Unsloth installation.
- **Fallback:** Automatically switches to Gemini/Mock if GPU is unavailable.

**Usage:** Simply select **Runtime -> Run all**.

In [ ]:
%%bash
# @title 1. Setup Repository

# Clone or update the repository
if [ ! -d "/content/app" ]; then
    echo "📥 Cloning repository..."
    git clone https://github.com/TCTri205/Neuro-Symbolic_Software_Security.git /content/app
else
    echo "🔄 Updating repository..."
    cd /content/app && git pull origin main
fi

cd /content/app
echo "✅ Repository ready at: $(pwd)"
echo "📍 Current branch: $(git branch --show-current)"
echo "📌 Latest commit: $(git log -1 --oneline)"

In [ ]:
%%bash
# @title 2. Fix Unsloth Installation
echo "🔧 Cleaning Unsloth stack..."

# 1. Uninstall existing packages to prevent conflicts
pip uninstall -y unsloth unsloth_zoo 2>/dev/null || true

# 2. Remove cached files
rm -rf /usr/local/lib/python3.12/dist-packages/unsloth* 2>/dev/null || true
rm -rf /usr/local/lib/python3.12/dist-packages/__pycache__/unsloth* 2>/dev/null || true

echo "✅ Cleanup complete!"

In [ ]:
%%bash
# @title 3. Install Dependencies
cd /content/app
echo "📦 Installing system dependencies..."
pip install -q -r requirements.txt

echo "📦 Installing Server Dependencies..."
pip install -q fastapi uvicorn pyngrok python-multipart nest-asyncio

echo "✅ System dependencies installed!"

In [ ]:
%%bash
# @title 4. Install Unsloth & GPU Optimizations

echo "📦 Installing core dependencies..."
pip install -q transformers>=4.51.3 datasets>=3.4.1 bitsandbytes>=0.45.5 peft>=0.18.0 accelerate>=0.34.1

echo "📦 Installing TRL..."
pip install -q "trl>=0.18.2,<0.25.0"

echo "📦 Installing Unsloth (No Deps)..."
pip install -q "unsloth @ git+https://github.com/unslothai/unsloth.git" --no-deps

echo "📦 Installing unsloth_zoo..."
pip install -q "unsloth_zoo>=2026.1.4"

echo "✅ Unsloth stack installed!"

In [ ]:
# @title 5. Configure & Launch Server
import uvicorn
import nest_asyncio
import os
from pyngrok import ngrok
from src.server.colab_server import app
from google.colab import userdata
import torch

# 1. Configure Provider
print("🔧 Configuring Provider...")
if torch.cuda.is_available():
    try:
        from unsloth import FastLanguageModel
        os.environ['LLM_PROVIDER'] = 'local'
        os.environ['LOCAL_MODEL_PATH'] = 'unsloth/Qwen2.5-Coder-7B-Instruct-bnb-4bit'
        print("   ✅ Using LOCAL provider (GPU-accelerated)")
    except ImportError:
        print("   ⚠️ Unsloth not found. Falling back to Remote.")
        os.environ['LLM_PROVIDER'] = 'gemini'
else:
    print("   ⚠️ No GPU detected. Falling back to Remote.")
    os.environ['LLM_PROVIDER'] = 'gemini'

# 2. Set API Keys if needed
if os.environ['LLM_PROVIDER'] == 'gemini':
    try:
        os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
        print("   ✅ GEMINI_API_KEY configured")
    except:
        print("   ❌ GEMINI_API_KEY missing! Server will likely fail.")

# 3. Configure Ngrok
print("\n🔌 Configuring Ngrok...")
try:
    NGROK_TOKEN = userdata.get('NGROK_TOKEN')
    ngrok.set_auth_token(NGROK_TOKEN)
    print("   ✅ Ngrok token configured")
except:
    print("   ⚠️ NGROK_TOKEN not found in Secrets. Tunnel might fail.")

# 4. Launch
PORT = 8000
ngrok.kill()
public_url = ngrok.connect(PORT).public_url

print("=" * 50)
print(f"🚀 SERVER IS LIVE AT: {public_url}")
print("=" * 50)

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=PORT)